# Fort Lauderdale flooding

This notebooks assess all GPM overpasses from the convective systems that produced flooding in Fort Lauderdale on 2023/04/12.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
from datetime import datetime
from gprof_nn.plotting import create_equidistant_area

start_time = datetime(2023, 4, 12, 12)
end_time = datetime(2023, 4, 13, 12)
lon_ftl = -80.15
lat_ftl = 26.13
area = create_equidistant_area(lon_ftl, lat_ftl, 0.75e6, 2e3)

In [5]:
import pyvista as pv
pv.start_xvfb()

## Find GPROF overpasses

The code below searches through all GPROF result files and resamples them to the study area.

In [3]:
def resample_results(retrieval_data):
    lons = retrieval_data.longitude.data
    lats = retrieval_data.latitude.data
    swath = SwathDefinition(lats=lats, lons=lons)
    sp = retrieval_data.surface_precip.data
    sp[sp < 0] = np.nan
    sp[:, 0] = np.nan
    sp[:, -1] = np.nan
    sp_r = resample_nearest(
        swath,
        sp,
        area,
        radius_of_influence=50e3,
        fill_value=np.nan
    )

    if np.all(np.isnan(sp_r)):
        return None

    lat_mean = np.nanmean(lats, -1)
    lon_mean = np.nanmean(lons, -1)
    scan_ind = np.argmin(np.sqrt((-80 - lon_mean) ** 2 + (25 - lat_mean) ** 2))
    time = retrieval_data.scan_time.data[scan_ind]

    return xr.Dataset({
        "time": (("time",), [time]),
        "surface_precip": (("time", "y", "x"), sp_r[None])
    })

In [4]:
from pathlib import Path
from pansat.products import NetcdfProduct
from pansat.catalog import find_files
from shapely.geometry import Point
from shapely.validation import make_valid
from pyresample import SwathDefinition
from pyresample.kd_tree import resample_nearest

results_gprof_nn_1d = {}
ftld = Point(lon_ftl, lat_ftl)

data_path = Path("/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/")
satellites = [
    "noaa19",
    "metopb",
    "metopc",
    "npp",
    "noaa20",
    "gcomw1",
    "f16",
    "f17",
    "f18",
    "gmi",
]
for sat in satellites:
    prod = NetcdfProduct({
        "time": "scan_time"
    })
    files = find_files(
        prod,
        data_path / sat / "gprof_nn_1d"
    )
    overpasses = []
    for result_file in files:
        poly = make_valid(prod.get_spatial_coverage(result_file))
        if poly.contains(ftld):
            retrieval_data = xr.load_dataset(result_file.local_path)
            overpass = resample_results(retrieval_data)
            if overpass is None:
                continue
            overpasses.append(overpass)
    if len(overpasses) > 0:
        results_gprof_nn_1d[sat] = xr.concat(overpasses, "time")

/home/simon/.local/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
from pansat.products import NetcdfProduct
from pansat.catalog import find_files
from shapely.geometry import Point
from shapely.validation import make_valid
from pyresample import SwathDefinition
from pyresample.kd_tree import resample_nearest

results_gprof_nn_3d = {}

data_path = Path("/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/")
for sat in satellites:
    prod = NetcdfProduct({
        "time": "scan_time"
    })
    files = find_files(
        prod,
        data_path / sat / "gprof_nn_3d"
    )
    overpasses = []
    for result_file in files:
        poly = make_valid(prod.get_spatial_coverage(result_file))
        if poly.contains(ftld):
            retrieval_data = xr.load_dataset(result_file.local_path)
            overpass = resample_results(retrieval_data)
            if overpass is None:
                continue
            overpasses.append(overpass)
    if len(overpasses) > 0:
        results_gprof_nn_3d[sat] = xr.concat(overpasses, "time")

In [6]:
from pansat.products.satellite.gpm import l2a_gprof
from pansat.catalog import find_files
from shapely.geometry import Point
from shapely.validation import make_valid
from pyresample import SwathDefinition
from pyresample.kd_tree import resample_nearest

results_gprof = {}

data_path = Path("/pdata4/archive/GPM/")
paths = {
    "noaa19": "2A_NOAA19_V7",
    "noaa20": "2A_NOAA20_V7",
    "npp": "2A_ATMS_V7",
    "gcomw1": "2A_AMSR2_V7",
    "metopb": "2A_METOPB_V7",
    "metopc": "2A_METOPC_V7",
    "f16": "2A_F16_V7",
    "f17": "2A_F17_V7",
    "f18": "2A_F18_V7",
    "gmi": "2A_GMI_V7",
    
}
for sat in satellites:
    files = find_files(
        l2a_gprof,
        data_path / paths[sat] / "2304"
    )
    overpasses = []
    for result_file in files:
        poly = make_valid(l2a_gprof.get_spatial_coverage(result_file))
        if poly.contains(ftld):
            retrieval_data = l2a_gprof.open(result_file.local_path)
            overpass = resample_results(retrieval_data)
            if overpass is None:
                continue
            overpasses.append(overpass)
    if len(overpasses) > 0:
        results_gprof[sat] = xr.concat(overpasses, "time")

Print number of collocations found for all satellites to ensure that the same collocations are found for all retrievals.

In [7]:
for sat in satellites:
    if sat in results_gprof_nn_1d:
        print(results_gprof_nn_1d[sat].time.size, results_gprof_nn_3d[sat].time.size, results_gprof[sat].time.size)

4 4 4
4 4 3
4 4 4
3 4 3
4 4 4
3 3 3
2 2 2
4 4 4
3 3 3


# MRMS

MRMS files are downloaded if not found localle.

In [ ]:
from pathlib import Path
from pansat.products.ground_based.mrms import mrms_precip_rate

mrms_files = sorted(list(Path("/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/MRMS").glob("*grib2")))
if len(mrms_files) < 1:
    mrms_files = mrms_precip_rate.download(start_time, end_time)
    

Load MRMS files. This is quite slow so skip every 5th file to achieve 10 min resolution.

In [ ]:
mrms_data = []
for mrms_file in mrms_files[::5]:
    mrms_data.append(xr.open_dataset(mrms_file)[{"latitude": slice(2500, -300), "longitude": slice(4500, 5500)}])

### Resample MRMS data to area

In [ ]:
lats_mrms = mrms_data[0].latitude.data
lons_mrms = mrms_data[0].longitude.data - 360
lons_mrms, lats_mrms = np.meshgrid(lons_mrms, lats_mrms)
mrms_area = SwathDefinition(lats=lats_mrms, lons=lons_mrms)
mrms_data_r = []
for data in mrms_data:
    sp_r = resample_nearest(
        mrms_area,
        data.unknown.data,
        area,
        radius_of_influence=5e3,
        fill_value=np.nan
    )
    mrms_data_r.append(xr.Dataset({
        "time": ("time", [data.time.data]),
        "surface_precip": (("time", "y", "x"), sp_r[None])
    }))

In [ ]:
mrms_data_r = xr.concat(mrms_data_r, "time")
mrms_data_r.to_netcdf("/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/mrms.nc")

In [ ]:
mrms_data_r = xr.load_dataset("/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/mrms.nc")

# Plot overpasses

In [ ]:
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm
import matplotlib as mpl

def plot_overpass(
    time,
    satellite_name,
    mrms_data,
    gprof_data,
    gprof_nn_1d_data,
    prof_nn_3d_data,
    slices=None
):
    fig = plt.figure(figsize=(9, 8))
    gs = GridSpec(2, 3, width_ratios=[1.0, 1.0, 0.075], wspace=0.05, hspace=0.1)
    crs = area.to_cartopy_crs()
    norm = LogNorm(1e-1, 1e2)
    
    if slices is None:
        slices = (slice(0, None), slice(0, None))
    
    levels = np.logspace(-1, 2, 13)
    
    area_r = area[slices[0], slices[1]]
    ext = area_r.area_extent
    ext = (ext[0], ext[2], ext[1], ext[3])
    cmap = mpl.colormaps.get_cmap("magma")
    
    ax = fig.add_subplot(gs[0, 0], projection=crs)
    sp = np.maximum(mrms_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
    
    cs = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
    for c in cs.collections:
        c.set_rasterized(True)
    ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
    ax.coastlines(color="grey")
    ax.set_title("(a) MRMS", loc="left")
    scale_bar(ax, 100e3, textcolor="grey", parts=5, border_color="grey")
    
    mask = gprof_nn_1d_data.surface_precip.data[slices[0], slices[1]] >= 0.0
    
    ax = fig.add_subplot(gs[0, 1], projection=crs)
    sp = np.maximum(gprof_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
    sp[~mask] = np.nan
    
    cs = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
    for c in cs.collections:
        c.set_rasterized(True)
    ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
    ax.coastlines(color="grey")
    ax.set_title("(b) GPROF", loc="left")
    scale_bar(ax, 100e3, textcolor="grey", parts=5, border_color="grey")
    
    if not np.any(np.isfinite(sp)):
        return None
    
    ax = fig.add_subplot(gs[1, 0], projection=crs)
    sp = np.maximum(gprof_nn_1d_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
    cs = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
    for c in cs.collections:
        c.set_rasterized(True)
    ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
    ax.coastlines(color="grey")
    ax.set_title("(c) GPROF-NN 1D", loc="left")
    scale_bar(ax, 100e3, textcolor="grey", parts=5, border_color="grey")
    
    ax = fig.add_subplot(gs[1, 1], projection=crs)
    sp = np.maximum(gprof_nn_3d_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
    sp[~mask] = np.nan
    m = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
    for c in m.collections:
        c.set_rasterized(True)
    ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
    ax.coastlines(color="grey")
    ax.set_title("(d) GPROF-NN 3D", loc="left")
    scale_bar(ax, 100e3, textcolor="grey", parts=5, border_color="grey")
    
    ax = fig.add_subplot(gs[:, -1])
    plt.colorbar(m, label="Surface precipitation [mm h$^{-1}$]", cax=ax)
    
    fig.suptitle(f"{satellite_name}, {time.astype('datetime64[s]')}", fontweight="bold")
    return fig
    

In [ ]:
from pansat.time import to_datetime

sensor_names = {
    "noaa19": "MHS (NOAA19)",
    "noaa20": "ATMS (NOAA19)",
    "metopb": "MHS (METOPA)",
    "metopc": "MHS (METOPB)",
    "f16": "SSMIS (F16)",
    "f17": "SSMIS (F17)",
    "f18": "SSMIS (F18)",
    "npp": "ATMS (NPP)",
    "gcomw1": "AMSR2 (GCOM-W1)",
    "gmi": "GMI",
}
for sat in satellites:
    if sat not in results_gprof:
        continue
    n_times = results_gprof[sat].time.size
    for ind, time in enumerate(results_gprof[sat].time.data):
        if (time > mrms_data_r.time[0]) and (time < mrms_data_r.time[-1]):
            mrms_data_t = mrms_data_r.interp(time=time)
            gprof_data = results_gprof[sat][{"time": ind}]
            gprof_nn_1d_data = results_gprof_nn_1d[sat].interp(time=time, method="nearest")
            gprof_nn_3d_data = results_gprof_nn_3d[sat].interp(time=time, method="nearest")
            fig = plot_overpass(time, sensor_names[sat], mrms_data_t, gprof_data, gprof_nn_1d_data, gprof_nn_3d_data, slices=(slice(100, -100), slice(100, -100)))
            if fig is None:
                continue
            pytime = to_datetime(time)
            timestr = pytime.strftime("%Y%m%d%H%M%S")
            fig.savefig(
                f"/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/results/{timestr}_{sat}.pdf",
                bbox_inches="tight"
            )
            del fig

# GPROF-NN HR profiles

Creates a 3D model of the GPROF-NN HR retrieval results.

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
img = np.array(Image.open("/home/simon/images/world.topo.bathy.200404.3x21600x10800.jpg"))

In [ ]:
from pyresample import create_area_def
from pyresample.kd_tree import resample_nearest

bm_area = create_area_def(
    "bm",
    {'proj': 'longlat', 'datum': 'WGS84'},
    shape=img.shape[:-1],
    area_extent=(-180, -90, 180, 90)
)
img_r = resample_nearest(
    bm_area,
    img,
    area,
    radius_of_influence=5e3
)

In [ ]:
import pyvista as pv
from gprof_nn.plotting import add_surface, add_hydrometeors, add_swath_edges
pv.set_jupyter_backend("pythreejs")
scene = pv.Plotter()
add_surface(scene, area, img_r)
filename = "/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/gprof_nn_hr.nc"
rwc_r, swc_r = add_hydrometeors(scene, area, filename, opacity=0.5)

In [ ]:
scene.export_vtkjs("/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/gprof_nn_hr")

# GPROF GMI precipitation

In [ ]:
hr_filename = "/gdata1/simon/gprof_nn/case_studies/fort_lauderdale/gprof_nn_hr.nc"
results_hr = resample_results(xr.load_dataset(hr_filename))
time = results_hr.time[0].data
mrms_data = mrms_data_r.interp(time=time)
gprof_data = results_gprof["gmi"][{"time": 0}]
gprof_nn_1d_data = results_gprof_nn_1d["gmi"][{"time": 0}]
gprof_nn_3d_data = results_gprof_nn_3d["gmi"][{"time": 0}]
gprof_nn_hr_data = results_hr[{"time": 0}]

In [ ]:
from gprof_nn.plotting import scale_bar
from matplotlib.gridspec import GridSpec
from matplotlib.colors import Normalize
import matplotlib as mpl

fig = plt.figure(figsize=(12, 8))
gs = GridSpec(2, 4, width_ratios=[1.0, 1.0, 1.0, 0.075], wspace=0.05, hspace=0.1)
crs = area.to_cartopy_crs()
norm = Normalize(0, 50)

slices = (slice(150, -150), slice(150, -150))
if slices is None:
    slices = (slice(0, None), slice(0, None))

levels = np.linspace(0, 50, 11)

area_r = area[slices[0], slices[1]]
ext = area_r.area_extent
ext = (ext[0], ext[2], ext[1], ext[3])
cmap = mpl.colormaps.get_cmap("magma")

ax = fig.add_subplot(gs[0, 0], projection=crs)
sp = np.maximum(mrms_data.surface_precip.data, 1e-2)[slices[0], slices[1]]

cs = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
for c in cs.collections:
    c.set_rasterized(True)
handles = ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua", label="Fort Lauderdale")
ax.coastlines(color="grey")
ax.set_title("(a) MRMS", loc="left")
scale_bar(ax, 50e3, textcolor="w", parts=5, border_color="w")

mask = gprof_nn_1d_data.surface_precip.data[slices[0], slices[1]] >= 0.0

ax = fig.add_subplot(gs[0, 1], projection=crs)
sp = np.maximum(gprof_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
print(sp.mean())
sp[~mask] = np.nan

cs = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
for c in cs.collections:
    c.set_rasterized(True)
ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
ax.coastlines(color="grey")
ax.set_title("(b) GPROF", loc="left")
scale_bar(ax, 50e3, textcolor="w", parts=5, border_color="w")

ax = fig.add_subplot(gs[0, 2], projection=crs)
sp = np.maximum(gprof_nn_1d_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
print(sp.mean())
cs = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
for c in cs.collections:
    c.set_rasterized(True)
ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
ax.coastlines(color="grey")
ax.set_title("(c) GPROF-NN 1D", loc="left")
scale_bar(ax, 50e3, textcolor="w", parts=5, border_color="w")

ax = fig.add_subplot(gs[1, 0], projection=crs)
sp = np.maximum(gprof_nn_3d_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
print(sp.mean())
sp[~mask] = np.nan
m = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
for c in m.collections:
    c.set_rasterized(True)
ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
ax.coastlines(color="grey")
ax.set_title("(d) GPROF-NN 3D", loc="left")
scale_bar(ax, 50e3, textcolor="w", parts=5, border_color="w")

ax = fig.add_subplot(gs[1, 1], projection=crs)
sp = np.maximum(gprof_nn_hr_data.surface_precip.data, 1e-2)[slices[0], slices[1]]
print(sp.mean())
sp[~mask] = np.nan
m = ax.contourf(sp[::-1], extent=ext, cmap=cmap, norm=norm, levels=levels, extend="both")
for c in m.collections:
    c.set_rasterized(True)
ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
ax.coastlines(color="grey")
ax.set_title("(e) GPROF-NN HR", loc="left")
scale_bar(ax, 50e3, textcolor="w", parts=5, border_color="w")

ax = fig.add_subplot(gs[1, 2])
ax.set_axis_off()
ax.legend(handles=[handles], loc="center")

ax = fig.add_subplot(gs[:, -1])
plt.colorbar(m, label="Surface precipitation [mm h$^{-1}$]", cax=ax)

fig.suptitle(f"GMI, {time.astype('datetime64[s]')}", fontweight="bold")
fig.savefig("gmi_overpass.png", bbox_inches="tight")


### L1C Data

In [ ]:
from pansat.products.satellite.gpm import l1c_gpm_gmi_r
l1c_data = l1c_gpm_gmi_r.open("/pdata4/archive/GPM/1CR_GMI_V7/2304/230412/1C-R.GPM.GMI.XCAL2016-C.20230412-S185045-E202315.051822.V07A.HDF5")

In [ ]:
fig = plt.figure(figsize=(9, 4))
gs = GridSpec(1, 3, width_ratios=[1.0, 1.0, 0.075])

area_s = area[150:-150, 150:-150]
ext = area_s.area_extent
ext = (ext[0], ext[2], ext[1], ext[3])

norm = Normalize(100, 300)
ax = fig.add_subplot(gs[0, 0], projection=crs)
ax.imshow(tbs_r[150:-150, 150:-150, -2], norm=norm, extent=ext, cmap="coolwarm", rasterized=True)
ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
ax.coastlines(color="grey")
ax.set_title("187 +/- 3 GHz")

ax = fig.add_subplot(gs[0, 1], projection=crs)
m = ax.imshow(tbs_r[150:-150, 150:-150, -1], norm=norm, extent=ext, cmap="coolwarm", rasterized=True)
ax.scatter([lon_ftl], [lat_ftl], marker="x", s=50, c="aqua")
ax.coastlines(color="grey")
ax.set_title("187 +/- 7 GHz")

ax = fig.add_subplot(gs[0, -1])
plt.colorbar(m, cax=ax, label="T$_B$ [K]")
fig.savefig("tbs.png", bbox_inches="tight")